# Importing the relavant modules

In [2]:
import itertools
import pandas as pd

from sklearn import (
    model_selection, linear_model, metrics, feature_selection, pipeline, base, neighbors, naive_bayes, tree
)

# Score prediction

## loading data

In [3]:
df1 = 'AimoScore_WeakLink_big_scores.xls'
df1 = pd.read_excel(df1)
df1

,AimoScore,No_1_Angle_Deviation,No_2_Angle_Deviation,No_3_Angle_Deviation,No_4_Angle_Deviation,No_5_Angle_Deviation,No_6_Angle_Deviation,No_7_Angle_Deviation,No_8_Angle_Deviation,No_9_Angle_Deviation,...,No_19_NASM_Deviation,No_20_NASM_Deviation,No_21_NASM_Deviation,No_22_NASM_Deviation,No_23_NASM_Deviation,No_24_NASM_Deviation,No_25_NASM_Deviation,No_1_Time_Deviation,No_2_Time_Deviation,EstimatedScore
0,0.323667,0.538020,0.815878,0.346724,0.382114,0.302248,0.947872,0.275945,0.521760,0.457198,...,0.833094,0.656624,0.642276,0.552846,0.648972,0.578192,0.560019,0.821616,0.818747,0.209947
1,0.323699,0.443807,0.306552,0.823529,0.188905,0.497370,0.140124,0.664275,0.521760,0.729316,...,0.670971,0.721186,0.826399,0.805356,0.848876,0.889048,0.816834,0.307987,0.248207,0.457198
2,0.848327,0.603539,0.373984,0.346724,0.590626,0.341942,0.298900,0.276901,0.623625,0.658058,...,0.670971,0.656624,0.642276,0.690579,0.648972,0.578192,0.555715,0.218556,0.235294,0.107126
3,0.351332,0.484935,0.623625,0.380201,0.975132,0.509326,0.888570,0.363462,0.847441,0.237207,...,0.953611,0.656624,0.642276,0.552846,0.648972,0.578192,0.744620,0.458154,0.432807,0.612626
4,0.627181,0.860832,0.657580,0.745576,0.552846,0.375897,0.483022,0.388331,0.521760,0.387853,...,0.670971,0.656624,0.642276,0.552846,0.648972,0.578192,0.308943,0.805356,0.774271,0.153515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089,0.407197,0.279770,0.131038,0.346724,0.317073,0.945481,0.646581,0.913439,0.521760,0.570540,...,0.670971,0.945002,0.642276,0.596365,0.648972,0.749880,0.943568,0.684840,0.711621,0.877571
2090,0.965930,0.564802,0.311813,0.376375,0.188905,0.187948,0.347202,0.227642,0.521760,0.248685,...,0.670971,0.753228,0.642276,0.552846,0.648972,0.869440,0.316117,0.148733,0.151124,0.030129
2091,0.527640,0.404489,0.214422,0.822827,0.937440,0.739255,0.909265,0.681471,0.658071,0.822350,...,0.973257,0.886819,0.640401,0.525310,0.776982,0.822827,0.367717,0.584050,0.570201,0.613658
2092,0.407197,0.606972,0.863897,0.467526,0.225406,0.946036,0.706781,0.934097,0.303247,0.576409,...,0.941738,0.886819,0.640401,0.725883,0.757402,0.822827,0.943649,0.953200,0.956543,0.761700


## applying the train test split

In [4]:
X1 = df1.drop(['AimoScore','EstimatedScore'], axis= 1)
Y1 = df1['AimoScore']

trainX1, testX1, trainY1, testY1 = model_selection.train_test_split(X1, Y1, test_size= 0.2)

## training different models

### Linear Reggression

In [5]:
model = linear_model.SGDRegressor()
model.fit(trainX1,trainY1)

testP1 = model.predict(testX1)
metrics.mean_squared_error(testY1,testP1)

0.0355558592714245

### Linear Regression with feature selection

In [6]:
model = pipeline.Pipeline([
    ('1',feature_selection.VarianceThreshold(threshold= 0.01)),
    ('2',linear_model.SGDRegressor())
])

model.fit(trainX1,trainY1)

testP1 = model.predict(testX1)
metrics.mean_squared_error(testY1,testP1)

0.036784186544916476

### Weighted linear regression

In [7]:
class WeightApplicator(base.BaseEstimator, base.TransformerMixin):
    weights = [
        1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 
        2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 
        1, 1, 2, 4, 4, 2, 2, 2, 2, 2, 
        1, 1, 1, 2, 2, 2, 2, 2, 1, 1,
    ]
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Perform arbitary transformation
        X * self.weights
        return X
    


model = pipeline.Pipeline([
    ('1',WeightApplicator()),
    ('2',linear_model.SGDRegressor())
])

model.fit(trainX1,trainY1)

testP1 = model.predict(testX1)
metrics.mean_squared_error(testY1,testP1)

0.03370139874541492

# weakest link classification

## loading the data

In [8]:
df2 = '20190108 scores_and_weak_links.xlsx'
df2 = pd.read_excel(df2)
df2

c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,ID,Date,SCORE,ForwardHead,LeftArmFallForward,RightArmFallForward,LeftShoulderElevation,RightShoulderElevation,ExcessiveForwardLean,LeftAsymmetricalWeightShift,RightAsymmetricalWeightShift,LeftKneeMovesInward,RightKneeMovesInward,LeftKneeMovesOutward,RightKneeMovesOutward,LeftHeelRises,RightHeelRises
0,cf0e95ab-d6ec-475a-9cc0-c9444b8dd080.Kinect,2018-07-19,0.612532,0.31,1.34,1.15,0.33,0.49,0.00,0.00,0.08,0.00,0.00,0.00,0.77,0.00,0.0
1,6210cc6d-575d-4e9f-bab1-4fe1da2a30a3.Kinect,2018-07-30,0.388809,1.49,1.42,1.39,0.00,0.00,1.08,0.10,0.00,0.00,0.00,0.00,0.40,0.00,0.0
2,2fc2eaad-c098-4b91-94c8-40241da1908e.Kinect,2018-06-19,0.205311,0.01,1.02,1.50,1.07,0.61,0.00,1.50,0.00,0.00,0.95,1.18,0.00,0.00,0.0
3,60ccb803-a949-403b-80d0-80f8aa36c60d.Kinect,2018-08-25,0.073400,0.00,1.01,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.00,1.36,1.50,0.21,1.3
4,90a7047d-452c-4889-b985-3f99afbb9ec0.Kinect,2018-05-09,0.728731,0.00,0.00,0.00,1.35,1.31,0.05,0.00,0.00,0.08,0.62,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2091,c2cd9a29-5ab2-4763-9e48-35fe4b2d82f9.Kinect,2018-07-19,0.717855,1.02,1.30,1.09,0.36,0.41,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.0
2092,75022edf-bc9d-4974-a178-28eaf9bf7d41.Kinect,2018-07-20,0.348149,1.50,1.32,1.39,0.56,0.03,0.00,1.50,0.00,0.00,0.02,0.00,0.00,0.00,0.0
2093,6e75d92b-8b71-476c-8323-5c19459cede2.Kinect,2018-05-31,0.754710,0.02,0.00,0.00,0.14,1.02,0.14,0.23,0.00,0.00,0.30,0.00,0.00,0.00,0.0
2094,743d9af8-cb6b-4769-99b3-e38840333e1d.Kinect,2018-06-26,0.545329,0.00,1.18,1.04,0.15,0.00,0.00,0.00,0.00,0.00,0.00,1.50,0.39,0.05,0.0


## applying train test split

In [9]:
X2 = df2.drop(['ID','Date','SCORE'], axis= 1)
Y2 = X2.values.argmax(axis= 1)

trainX2, testX2, trainY2, testY2 = model_selection.train_test_split(X2, Y2, test_size= 0.2)

## training models

### Nearest neighbours

In [10]:
model = neighbors.KNeighborsClassifier(n_neighbors= 10)

model.fit(trainX2,trainY2)

testP2 = model.predict(testX2)
metrics.accuracy_score(testY2,testP2)

0.6738095238095239

### Naive Bayes

In [11]:
model = naive_bayes.GaussianNB()

model.fit(trainX2,trainY2)

testP2 = model.predict(testX2)
metrics.accuracy_score(testY2,testP2)

0.2904761904761905

### logistic regressor

In [12]:
model = linear_model.LogisticRegression()

model.fit(trainX2,trainY2)

testP2 = model.predict(testX2)
metrics.accuracy_score(testY2,testP2)

c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8404761904761905

# Improving models using cross validation

## champion regression model

In [13]:
losses    = ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']
penalties = ['l2', 'l1', 'elasticnet', None]

loss, penalty = max(
    itertools.product(losses,penalties), 
    key= lambda e: model_selection.cross_val_score(
        linear_model.SGDRegressor(loss= e[0], penalty= e[1]), 
        trainX1, 
        trainY1
    ).mean()
)

print(f'the maximum score is achieved with loss {loss} and penalty {penalty}')

the maximum score is achieved with loss epsilon_insensitive and penalty None


In [20]:
model = linear_model.SGDRegressor(loss= 'epsilon_insensitive', penalty= 'l2').fit(trainX1,trainY1)
model.fit(trainX1,trainY1)

testP1 = model.predict(testX1)
metrics.mean_squared_error(testY1,testP1)

0.025296526707442354

## champion classsification model

In [15]:
penalties = ['l2', 'l1', None]

penalty = max(
    penalties, 
    key= lambda e: model_selection.cross_val_score(
        linear_model.LogisticRegression(penalty= e, solver= 'saga'), 
        trainX2, 
        trainY2
    ).mean()
)

print(f'the maximum score is achieved with penalty {penalty}')

c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\hahme\min

the maximum score is achieved with penalty l1


c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [22]:
model = linear_model.LogisticRegression(penalty= 'l1', solver= 'saga')
model.fit(trainX2,trainY2)

testP2 = model.predict(testX2)
metrics.accuracy_score(testY2,testP2)

c:\Users\hahme\miniconda3\envs\fozan\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.9238095238095239

# Tree based approach

## regression model

In [19]:
model = tree.DecisionTreeRegressor()
model.fit(trainX1,trainY1)

testP1 = model.predict(testX1)
metrics.mean_squared_error(testY1,testP1)

0.03956074584487774

## classification model

In [18]:
model = tree.DecisionTreeClassifier()
model.fit(trainX2,trainY2)

testP2 = model.predict(testX2)
metrics.accuracy_score(testY2,testP2)

0.8428571428571429